## Functions to analyse freqsweep, surprise, and convergence protocols

In [ ]:
import sys
import os
import importlib
from pathlib import Path
import pickle

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.signal import find_peaks
import plotly.express as px
import plotly.graph_objects as go


%matplotlib widget
# sns.set_context('talk')

%tb
from eidynamics import utils, data_quality_checks, ephys_classes, abf_to_data
import parse_data
import all_cells

In [ ]:
cellpath = Path(r"C:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Data\23-06-07_GikeAA313\3131.pkl")

In [ ]:
cell = ephys_classes.Neuron.loadCell(cellpath)

In [ ]:
cell.data.keys()

In [ ]:
cell.data

In [ ]:
freqdf = cell.data['FreqSweep']
surprisedf = cell.data['surprise']

In [ ]:
surprisedf.iloc[0,:24]

### Things to calculate
1. first pulse response: peak, slope, auc
2. PPR vs FPR

### plot all the response with respect to numSq

In [ ]:
df = surprisedf[ surprisedf["exptID"] != 5]
slicevar = 'exptID'

sqs = np.unique(df[slicevar])
fig, axs = plt.subplots(len(sqs), 1, figsize=(12,9), sharex=True)
for i,s in enumerate(sqs):
    print(slicevar, ": ", s, i)
    dftemp = df[ (df[slicevar]==s) ]
    window = 20023+utils.epoch_to_datapoints([0, 1.0])
    m = dftemp.iloc[:, window].to_numpy().T
    pd = np.average(dftemp.iloc[:, window+40000].to_numpy().T, axis=1)
    t = x = np.arange(0, 1.0, 1 / 2e4)
    axs[i].plot(window, m, 'gray', label=str(s))
    axs[i].set_ylim([-2, 10])
    
    axs[i].plot(window, np.average(m, axis=1), 'black', label=str(s))
    axs[i].plot(window, pd , 'red', label=str(s))
    axs[i].set_title(str(s))

fig.show()

In [ ]:
df

In [ ]:
df = freqdf#[ freqdf["exptID"] != 5]
slicevar = 'exptID'

sqs = np.unique(df[slicevar])
fig, axs = plt.subplots(len(sqs), 1, figsize=(12,9), sharex=True)
for i,s in enumerate(sqs):
    print(slicevar, ": ", s, i)
    dftemp = df[ (df[slicevar]==s) ]
    window = 23+utils.epoch_to_datapoints([0, 1.0])
    m = dftemp.iloc[:, window].to_numpy().T
    pd = np.average(dftemp.iloc[:, window+40000].to_numpy().T, axis=1)
    t = x = np.arange(0, 1.0, 1 / 2e4)
    axs[i].plot(window, m, 'gray', label=str(s))
    axs[i].set_ylim([-5, 20])
    
    axs[i].plot(window, np.average(m, axis=1), 'black', label=str(s))
    axs[i].plot(window, pd , 'red', label=str(s))
    axs[i].set_title(str(s))

fig.show()

In [ ]:
from eidynamics import plot_tools

In [ ]:
import importlib
importlib.reload(plot_tools)

In [ ]:
plot_tools.plot_data_from_df(freqdf[freqdf["condition"]=='Control'], start = 23, simplify=False, combine=False, fig=None, ax=None, exclude_from_simplification=[])

### Spike Train

#### STA

In [ ]:
def plot_spike_train_STA(data, squares='all', save_fig=True):
    cell = np.array(data[0][0])
    led = np.array(data[0][2])
    if squares=5:
        numSweeps = np.array([1,9,17, 2,10,18, 3,11,19, 4,12,20, 5,13,21])-1
    elif squares=15:
        numSweeps = np.array([6,14,21, 7,15,22, 8,16,23])-1
    elif squares = 'all':
        numSweeps = np.arange(0,24)

    fig, ax = plt.subplots(1,1,figsize=(10,10))
    mean_sta = np.zeros((len(numSweeps), 2000))
    for n, s in enumerate(numSweeps):
        trial_cell = cell[s,:]
        trial_stim = led[s,:]
        peaks, _ = find_peaks(trial_cell, height=50, distance=100)
        numEvents = len(peaks)
        pre_event_stim = np.zeros((numEvents,2000))
        print(n, s, numEvents)
        for i in range(numEvents):
            pre_event_stim[i,:] = trial_stim[peaks[i]-2000:peaks[i]]
            # draw individual stim slices
            # ax.plot(np.linspace(-100, 0, 2000), pre_event_stim[i,:], color='k', alpha=0.05)
        mean_sta[n,:] = np.mean(pre_event_stim, axis=0)
        # make a filled plot of mean sta
        ax.fill_between(np.linspace(-100, 0, 2000), np.mean(pre_event_stim, axis=0), color='k', alpha=0.2)

    ax.set_xlabel('Time before a spike event (ms)')
    ax.set_ylabel('LED intensity (a.u.)')
    ax.set_title( 'Spike Triggered Average for {} sq patterns'.format(squares) )

    # show figure and save
    if save_fig:
        filename1 = 'sta_{}sq.svg'.format(squares)
        filename2 = 'sta_{}sq.png'.format(squares)
        plt.savefig(filename1)
        plt.savefig(filename2)


In [ ]:
train_ctrl_data = train_ctrl_data = abf_to_data.abf_to_data(spiketrain_ctrl_path, filter_type='bessel', filter_cutoff=2000,data_order="channelwise", sweep_baseline_epoch=[0, 0.2], baseline_subtraction=True)

Loading ABF file
Datafile has 24 sweeps in 6 channels.


In [ ]:
plot_spike_train_STA(train_ctrl_data, squares='all', save_fig=False)